# Set up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import os.path as path

root_path = "/content/drive/MyDrive/DFS/pu/thesis/the_new_one"

data_path = path.join(root_path, 'data')
model_path = path.join(root_path, 'models')

!ls $data_path
!ls $model_path

df_aspect.csv		 df_preprocess.csv		  filtered_data.csv
df_aspect_stopwords.csv  df_preprocess_stopwords.csv	  image
df_none.csv		 df_with_aspect_sentiment.csv	  old
df_none_none.csv	 df_with_aspect_sentiment.gsheet  short_data.csv
CafeBERT  phobert-base	phobert-base-v2  visobert


# Load data

In [3]:
import pandas as pd
df = pd.read_csv(path.join(data_path, 'df_none.csv'))
df

,place_id,name,status,location,overall_rating,price,rating,review_time,comment,street_number,route,area,a_food,a_service,a_atmosphere,type,clean_cmt
0,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,5.0,2024-11-01,🌟🌟🌟🌟🌟 Mình rất hài lòng khi ghé Dominos Pizza ...,7,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,<emoji> glowing star </emoji> <emoji> glowing ...
1,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2024-11-01,Cỡ 9 có thể 2ng ăn sức ăn mạnh chắc 1 người ăn,7,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,cỡ chín ngàn có_thể hai ngàn ng ăn sức ăn mạnh...
2,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2022-04-01,Pizza ăn ngon nhưng món mì nên bỏ khỏi thực đơ...,7,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,pizza ăn ngon nhưng món mì nên bỏ khỏi thực_đơ...
3,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2019-10-01,địa điểm dễ tìm ngay đầyou đường phan huy ích ...,7,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,địa_điểm dễ tìm ngay đầu đường phan huy ích vs...
4,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2019-04-01,Phục vụ Ok Món ăn ổn Không gian cũng gọn và lị...,7,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,phục_vụ ok món ăn ổn không_gian cũng gọn và lị...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,ChIJKw5ogJwpdTERm3eYnSlmsnw,BA GÁC Vietnamese Grill & Beer Garden Quang Trung,OPERATIONAL,NaN,4.7,NaN,1.0,2023-01-01,Đề nghị quản lý nhà hàng nên xem xét lại cách ...,1,Đ. Quang Trung,Gò Vấp,NaN,Ăn tại chỗ,NaN,Bữa tối,đề_nghị quản_lý nhà_hàng nên xem_xét lại cách ...
8322,ChIJVVVVVTwqdTERRmcHe-hzkms,JUNE Noodle House NAT,OPERATIONAL,NaN,4.5,1-100.000,1.0,2023-01-01,Dạ nay ăn xong tô mì cay thì gặp them này,92/3,D. Vạn Hạnh 3,Hóc Môn,NaN,Ăn tại chỗ,NaN,Bữa tối,dạ nay ăn xong tô mì cay thì gặp them này
8323,ChIJN1syps8vdTERs5a9qK1_OBg,GoGi House Âu Cơ,OPERATIONAL,NaN,4.7,200-300,5.0,2022-09-01,đắt xắt ra miếng từ món ăn chất lượng đến dịch...,303,Đ. Âu Cơ,Tân Phú,NaN,Ăn tại chỗ,NaN,Bữa tối,đắt_xắt_ra_miếng từ món ăn chất_lượng đến dịch...
8324,ChIJN1syps8vdTERs5a9qK1_OBg,GoGi House Âu Cơ,OPERATIONAL,NaN,4.7,200-300,5.0,2024-10-01,CB 199k,303,Đ. Âu Cơ,Tân Phú,5.0,NaN,NaN,NaN,cb 199 ngàn


# Inference

In [4]:
model_name = "uitnlp/CafeBERT"

In [5]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch

class AspectBasedSentimentModel(nn.Module):
    def __init__(self, model_name, max_length=128):
        super(AspectBasedSentimentModel, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size

        self.food_head = nn.Linear(hidden_size, 5)
        self.service_head = nn.Linear(hidden_size, 5)
        self.atmosphere_head = nn.Linear(hidden_size, 5)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def tokenize(self, text):
        return self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        food_logits = self.food_head(cls_output)
        service_logits = self.service_head(cls_output)
        atmosphere_logits = self.atmosphere_head(cls_output)
        return food_logits, service_logits, atmosphere_logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model = AspectBasedSentimentModel(model_name)  # Initialize the model structure
model.load_state_dict(torch.load(path.join(model_path, model_name.split("/")[-1], 'model.pt'), map_location=torch.device(device)))  # Load weights
model.eval()  # Set the model to evaluation mode
model.to(device)  # Ensure the model is on the correct device

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at uitnlp/CafeBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

<ipython-input-6-40477595aa8b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path.join(model_path, model_name.split("/")[-1], 'model.pt')

AspectBasedSentimentModel(
  (encoder): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features

In [7]:
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np

# Parameters
batch_size = 16  # Adjust based on your GPU memory

# Tokenize text and create batches
tokens = model.tokenize(list(df['clean_cmt']))
input_ids = torch.tensor(tokens['input_ids'])
attention_mask = torch.tensor(tokens['attention_mask'])

dataset = TensorDataset(input_ids, attention_mask)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Store predictions
food_preds, service_preds, atmosphere_preds = [], [], []

# Perform batch predictions
model.eval()
with torch.no_grad():
    for batch in dataloader:
        batch_input_ids, batch_attention_mask = batch
        batch_input_ids = batch_input_ids.to(device)
        batch_attention_mask = batch_attention_mask.to(device)

        food_logits, service_logits, atmosphere_logits = model(batch_input_ids, batch_attention_mask)

        # Convert logits to predictions
        food_preds.append(torch.argmax(food_logits, dim=-1).cpu().numpy())
        service_preds.append(torch.argmax(service_logits, dim=-1).cpu().numpy())
        atmosphere_preds.append(torch.argmax(atmosphere_logits, dim=-1).cpu().numpy())

# Concatenate all predictions
df['a_food'] = np.concatenate(food_preds)
df['a_service'] = np.concatenate(service_preds)
df['a_atmosphere'] = np.concatenate(atmosphere_preds)

df

<ipython-input-7-2eb42f6fca9a>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens['input_ids'])
<ipython-input-7-2eb42f6fca9a>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens['attention_mask'])


,place_id,name,status,location,overall_rating,price,rating,review_time,comment,street_number,route,area,a_food,a_service,a_atmosphere,type,clean_cmt
0,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,5.0,2024-11-01,🌟🌟🌟🌟🌟 Mình rất hài lòng khi ghé Dominos Pizza ...,7,Phan Huy Ích,Tân Bình,4,4,4,NaN,<emoji> glowing star </emoji> <emoji> glowing ...
1,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2024-11-01,Cỡ 9 có thể 2ng ăn sức ăn mạnh chắc 1 người ăn,7,Phan Huy Ích,Tân Bình,3,3,3,NaN,cỡ chín ngàn có_thể hai ngàn ng ăn sức ăn mạnh...
2,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2022-04-01,Pizza ăn ngon nhưng món mì nên bỏ khỏi thực đơ...,7,Phan Huy Ích,Tân Bình,3,3,3,NaN,pizza ăn ngon nhưng món mì nên bỏ khỏi thực_đơ...
3,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2019-10-01,địa điểm dễ tìm ngay đầyou đường phan huy ích ...,7,Phan Huy Ích,Tân Bình,3,3,3,NaN,địa_điểm dễ tìm ngay đầu đường phan huy ích vs...
4,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,NaN,4.0,2019-04-01,Phục vụ Ok Món ăn ổn Không gian cũng gọn và lị...,7,Phan Huy Ích,Tân Bình,4,4,4,NaN,phục_vụ ok món ăn ổn không_gian cũng gọn và lị...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,ChIJKw5ogJwpdTERm3eYnSlmsnw,BA GÁC Vietnamese Grill & Beer Garden Quang Trung,OPERATIONAL,NaN,4.7,NaN,1.0,2023-01-01,Đề nghị quản lý nhà hàng nên xem xét lại cách ...,1,Đ. Quang Trung,Gò Vấp,2,0,0,Bữa tối,đề_nghị quản_lý nhà_hàng nên xem_xét lại cách ...
8322,ChIJVVVVVTwqdTERRmcHe-hzkms,JUNE Noodle House NAT,OPERATIONAL,NaN,4.5,1-100.000,1.0,2023-01-01,Dạ nay ăn xong tô mì cay thì gặp them này,92/3,D. Vạn Hạnh 3,Hóc Môn,3,0,3,Bữa tối,dạ nay ăn xong tô mì cay thì gặp them này
8323,ChIJN1syps8vdTERs5a9qK1_OBg,GoGi House Âu Cơ,OPERATIONAL,NaN,4.7,200-300,5.0,2022-09-01,đắt xắt ra miếng từ món ăn chất lượng đến dịch...,303,Đ. Âu Cơ,Tân Phú,2,2,4,Bữa tối,đắt_xắt_ra_miếng từ món ăn chất_lượng đến dịch...
8324,ChIJN1syps8vdTERs5a9qK1_OBg,GoGi House Âu Cơ,OPERATIONAL,NaN,4.7,200-300,5.0,2024-10-01,CB 199k,303,Đ. Âu Cơ,Tân Phú,4,4,3,NaN,cb 199 ngàn


In [8]:
df['a_food'] = df['a_food'] + 1
df['a_service'] = df['a_service'] + 1
df['a_atmosphere'] = df['a_atmosphere'] + 1

In [9]:
df['a_food'].unique(), df['a_service'].unique(), df['a_atmosphere'].unique()

(array([5, 4, 3, 2, 1]), array([5, 4, 3, 1, 2]), array([5, 4, 3, 1, 2]))

In [10]:
df.to_csv(path.join(data_path, 'df_with_aspect_sentiment.csv'), index=False)